# MPC intro part 2
This introduction provides more practice for writing your integrator, cost functions and building MPC problems. 

This part drops the dependency on robot.py, i.e. no Pinocchio-CasADi is required and you should be able to run with a vanilla CasADi install.

In [3]:
# Imports, sane defaults
import time
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = True
plt.rcParams['image.aspect'] = 0.8

nlpopts = { "print_time":False,      # Prints the time for evaluating objective, calculating gradients, etc
            "ipopt.print_level":0,   # Integer from 0 to 5, 5 printing the most information
            "ipopt.tol":1.0e-5,
            "ipopt.acceptable_constr_viol_tol":2.0e-04,
            "ipopt.warm_start_init_point":"yes",
            "ipopt.warm_start_bound_frac":1.0e-09,
            "ipopt.warm_start_bound_push":1.0e-09,
          }

# Getting set up: Dynamics and Cost
Every MPC problem requires (1) dynamics, (2) a cost function, and optionally (3) constraints. It's good practice to write these as functions, to make the construction of the MPC problem easier to read and provide a single, obvious place where these functions are built.

We'll make a few assumptions here; the dynamic parameters are known and fixed, but the cost function will _always_ need tuning - it's good practice to load these tuning parameters from a YAML or similar such that the problem is (somewhat) repeatable. 

Let's consider dynamics of the form $M\ddot{x} + B\dot{x} + K(x-x_0) = u$, with input $u$ and state $\xi = (\dot{x}, x)$. First try writing an explicit Euler integrator, i.e. $xi_+ = xi + dt*\dot{xi}$.

Let's consider a cost function of the form $q_d\Vert x-x^d \Vert + q_v\Vert \dot{x} \Vert + r \Vert u \Vert$

In [ ]:
''' Returns a CasADi function from previous state and input to the next state. '''
def build_step(dt=0.05, M=1.5, B=5., K=50., x_0=0., fn_opts = {}) -> ca.Function:
    # TODO: write your integrator here
    
    
    step = ca.Function('step', [x, u], [x_next], ['x', 'u'], ['x_next'], fn_opts)
    return step
    

''' Returns a CasADi function from state and input to scalar '''
def build_cost(q_d=1., q_v=0.1, r=1.) -> ca.Function:
    # TODO: build a function from state to the control input
    
    return cost

# Test the dynamics
It's good practice to test that your dynamics is working as you expect, and probably you'd like to benchmark it. Some tests that I like to do: (1) check the unforced system converges as you expect (i.e. when $u=0$). (2) check that the input makes the expected change on the system. (3) timeit benchmarking to check the speed.

It's also usually good to get a feel for how sensitive the system is to step size.  If you notice oscillations, NaNs, or the state blows up, that means something is numerically unstable. It could mean the dynamics themselves are unstable, or that numerical issues occur with your integration scheme. 

What to do when its numerical instability? Typically reducing step size reduces the error of your integrator, but you might also need to do a different integrator. For example, in robotics we typically use a semi-implicit integrator, where $\dot{x}_+ = f(x, u)$ and $x_+ = x + \dot{x}_+$.  This is common when the differential equation is stiff (here, literally when $K$ is large). 

In [ ]:
step = build_step()
x = ca.DM.ones(2)
for _ in range(100):
    x = step(x, 0.0)
print(f"Final state: {x}")

u_test = [-1.0, 0, 1.0]
x0 = ca.DM.ones(2)
for u in u_test:
    x = step(x0, u)
    print(f"Input {u}\n  Initial state: {}\n    Final state: {}")
    
%timeit step(x0, 0.0)

# Build the MPC problem
Let's build the MPC problem where we take as parameter the initial state ($x_0$). Why define this as parameter? The state of our system will be different each time we start the problem, and we don't want to rebuild the optimization problem each time step. 

First try a multiple shooting problem, where we have create a symbolic state vector for each time step, and impose the constraint that the dynamics and next symbolic state are equal. Recall that we can impose equality constraints with $0<g<0$. 

In [ ]:
prob = {'x': , # Decision variables, your states and inputs
        'p': , # Parameters
        'f': , # Objective function
        'g': , # Constraint functions
       }

solver = ca.nlpsol('solver', 'ipopt', prob, nlpopts)
print(f"Built solver {solver}")

# Solve the MPC problem
To solve the MPC problem, we give numerical values to initialize our decision vector, the value of the parameters, and the constraints.

In [1]:

xi = ca.DM([0.5, 0.1])

args = {'x0': ,  # initial guess for the decision variables
        'p':  ,  # numerical value of the parameters
        'lbg': , # numerical value of the lower bound 
        'ubg': , # numerical value of the upper bound
       }

# Simuluate the system with step, and solve the MPC for each time step.
for _ in range(50):
    args['p'] = xi # update the initial state 
    sol = solver(**args)
    # TODO: extract the first control action from your MPC here
    u = 
    xi = step(xi, u)


SyntaxError: invalid syntax (847759705.py, line 3)